### Load data

In [7]:
import markdown
with open('../Data/Investor Reports/2012-02.md', 'r', encoding='utf-8') as file:
    content = file.read()

html = markdown.markdown(content)

In [61]:
print(html)

<h1>Everpix February 2012 Report</h1>
<h2>High Level</h2>
<ul>
<li>2 new hires on the Everpix team</li>
<li>Good engineering progress on all fronts</li>
<li>Surprisingly as many photos imported in February than January (15 millions)</li>
</ul>
<h2>Hiring Update</h2>
<p>We finally found a couple great candidates:</p>
<ul>
<li>Sameer Sundresh as Cloud Infrastructure Lead Engineer:</li>
<li>http://www.linkedin.com/in/sameersundresh</li>
<li>PhD in computer science, MemSQL, Rdio...</li>
<li>Hired on February 13th</li>
<li>George Leontiev as iOS Lead Engineer:</li>
<li>http://www.linkedin.com/pub/george-leontiev/43/a28/194</li>
<li>Apple engineer on iWork team</li>
<li>Hired on March 5th</li>
</ul>
<p>Ideally, we would only hire people who are clearly more talented and experienced than us in specific areas: that’s what we did for Jason Eberle, our web engineer, and our new hire Sameer Sundresh. But when it comes to iOS development, we already have a very strong in-house experience, so it wo

In [63]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings()

/Users/tanghao/NUS/Internship/EverpixAnalysis/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/tanghao/NUS/Internship/EverpixAnalysis/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [66]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [74]:
a='''Ideally, we would only hire people who are clearly more talented and experienced than us in specific areas: that’s what we did for Jason Eberle, our web engineer, and our new hire Sameer Sundresh. But when it comes to iOS development, we already have a very strong in-house experience, so it would be really difficult, very expensive and would take a long time to find such people. So instead, considering the core of the iOS app has been built, and most of the work is now focused on the UX layer (and later build the iPad version), we decided to look for a good iOS engineer for these specific tasks. In this very competitive market, we were fortunate enough to get George Leontiev before he really hit the market. His addition to the team will allow us to both continue building the iOS app at a good pace and also leave enough bandwidth to work on our image analysis research.</p>
'''

# embeddings = model.encode(a)
# print(embeddings)

In [75]:
len(a)

885

### KnowledgeGraph

In [12]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_entities_relations(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    relations = []
    for ent in doc.ents:
        for token in ent.root.head.subtree:
            if token.dep_ in ["nsubj", "dobj"]:
                relations.append((ent.text, token.dep_, token.text))
    return entities, relations


In [13]:
entities, relations = extract_entities_relations(html)

In [54]:
def create_triples(relations):
    triples = []
    for ent1, dep, ent2 in relations:
        triples.append((ent1, dep, ent2))
    return triples

triples = create_triples(relations)

In [56]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", auth=("neo4j", "Th503221."))

In [57]:
from py2neo import Node, Relationship

for subj, rel, obj in triples:
    subj_node = Node("Entity", name=subj)
    obj_node = Node("Entity", name=obj)
    relationship = Relationship(subj_node, rel, obj_node)
    graph.merge(subj_node, "Entity", "name")
    graph.merge(obj_node, "Entity", "name")
    graph.create(relationship)


In [58]:
# 查询所有节点
nodes = graph.run("MATCH (n) RETURN n")
for record in nodes:
    print(record)

# 查询特定关系
relationships = graph.run("MATCH (a)-[r:KNOWS]->(b) RETURN a, r, b")
for record in relationships:
    print(record)


Node('Entity', name='Alice')
Node('Entity', name='Bob')
Node('Entity', name='Acme Corp')
Node('Entity', name='Wonderland')
Node('Entity', name='Dreamland')
Node('Entity', name='February than January')
Node('Entity', name='photos')
Node('Entity', name='Sameer')
Node('Entity', name='Sundresh')
Node('Entity', name='George Leontiev')
Node('Entity', name='>')
Node('Entity', name='Leontiev')
Node('Entity', name='Apple')
Node('Entity', name='engineer')
Node('Entity', name='iPad')
Node('Entity', name='version')
Node('Entity', name='he')
Node('Entity', name='market')
Node('Entity', name='Everpix Alpha')
Node('Entity', name='infrastructure')
Node('Entity', name='10+ millions')
Node('Entity', name='10')
Node('Entity', name='photos.</li')
Node('Entity', name='<')
Node('Entity', name='li')
Node('Entity', name='3')
Node('Entity', name='2 months')
Node('Entity', name='website')
Node('Entity', name='which')
Node('Entity', name='month.</li')
Node('Entity', name='primary')
Node('Entity', name='goal')
No

In [59]:
entities = graph.nodes.match("Entity")
texts = []
for entity in entities:
    relationships = list(graph.match((entity,), r_type=None))
    for rel in relationships:
        relationship_text = f"{rel.start_node['name']} {rel.type} {rel.end_node['name']}"
        texts.append(relationship_text)

texts

['Alice <function Relationship.type at 0x1335b8fe0> Wonderland',
 'Alice <function Relationship.type at 0x1335b8fe0> Bob',
 'Bob <function Relationship.type at 0x1335b8fe0> Acme Corp',
 'Wonderland <function Relationship.type at 0x1335b8fe0> Dreamland',
 'February than January <function Relationship.type at 0x1335b8fe0> photos',
 'Sameer <function Relationship.type at 0x1335b8fe0> Sundresh',
 'George Leontiev <function Relationship.type at 0x1335b8fe0> market',
 'George Leontiev <function Relationship.type at 0x1335b8fe0> he',
 'George Leontiev <function Relationship.type at 0x1335b8fe0> Leontiev',
 'George Leontiev <function Relationship.type at 0x1335b8fe0> >',
 'Apple <function Relationship.type at 0x1335b8fe0> accounts',
 'Apple <function Relationship.type at 0x1335b8fe0> users',
 'Apple <function Relationship.type at 0x1335b8fe0> Apple',
 'Apple <function Relationship.type at 0x1335b8fe0> which',
 'Apple <function Relationship.type at 0x1335b8fe0> >',
 'Apple <function Relationshi

### Tree index

In [1]:
import markdown
from bs4 import BeautifulSoup

def parse_markdown(md_content):
    html = markdown.markdown(md_content)
    soup = BeautifulSoup(html, 'html.parser')
    data = []
    current_h1 = None
    current_h2 = None
    for element in soup.find_all(['h1', 'h2', 'p']):
        if element.name == 'h1':
            current_h1 = element.get_text()
        elif element.name == 'h2':
            current_h2 = element.get_text()
        elif element.name == 'p':
            data.append({
                'h1': current_h1,
                'h2': current_h2,
                'text': element.get_text()
            })
    return data


In [9]:
from llama_index.core import Node


ImportError: cannot import name 'Node' from 'llama_index' (unknown location)

In [4]:
from llama_index import TreeIndex, Node

def create_tree_index(parsed_data):
    root = Node("Root Node")
    h1_nodes = {}

    for entry in parsed_data:
        h1_text = entry['h1']
        h2_text = entry['h2']
        text = entry['text']

        if h1_text not in h1_nodes:
            h1_nodes[h1_text] = Node(h1_text, parent=root)
        h1_node = h1_nodes[h1_text]

        h2_node = Node(h2_text, parent=h1_node)
        Node(text, parent=h2_node)

    return TreeIndex(root)


ImportError: cannot import name 'GPTTreeIndex' from 'llama_index' (unknown location)